# **API-Einbindung mit Grok 3 Mini und NiceGUI**

In [2]:
# Installiere die benötigten Bibliotheken
!pip install xai-sdk

# Lade das deutsche Sprachmodell für spaCy (falls später für ABSA oder Syn-Chain benötigt)
!python -m spacy download de_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 84.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
!pip install mistralai

In [ ]:
!pip install ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00


## Import relevanter Bibliotheken

In [49]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import IntSlider, Button, FloatProgress, FileUpload, Textarea, VBox, Label, HBox, Output
from IPython.display import display
from io import BytesIO
import asyncio
import nest_asyncio
from xai_sdk import Client
from xai_sdk.chat import user, system
from mistralai import Mistral
import requests
from transformers import pipeline
from google.colab import userdata
import os
import json
import time
nest_asyncio.apply()


## Erstellung Client Objekts mit API Zugriff:

### API-Key in Google Colab als Umgebungsvariable setzen

In [9]:
os.environ['XAI_API_KEY'] = userdata.get('XAI_API_KEY')

print('API-Key geladen (letzte 4 Zeichen):', os.environ['XAI_API_KEY'][-4:])

API-Key geladen (letzte 4 Zeichen): NTQk


Systemprompt hinzufügen, um zu bestimmen, wie das Sprachmodell die Anfrage des Users verarbeiten soll:

In [10]:
# Erstellung Client Objekts mit API Zugriff:
client = Client(api_key=os.getenv("XAI_API_KEY")) # Annahme: API-Key wird als Umgebungsvariable gesetzt

chat = client.chat.create(model="grok-3-mini")  #Erstellung eines Chat-Objektes (global für Multi-Turn)
chat.append(system("You are Grok, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."))

messages {
  content {
    text: "You are Grok, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."
  }
  role: ROLE_SYSTEM
}
model: "grok-3-mini"

In [52]:
# Funktion für Grok Anfrage
def ask_grok(change=None):
    prompt = prompt_input.value.strip()
    if not prompt:
        output_label.value = "Please write in your prompt!"
        return
    try:
        output_label.value = "Grok is thinking..."
        chat.append(user(prompt))

        # synchrone Anfrage
        response = chat.sample()
        text = response.content if hasattr(response, 'content') else str(response)
        output_label.value = text

    except Exception as e:
        output_label.value = f"Fehler bei de API-Anfrage: {e}"

def uploader_callback(change):
  upload_button.description = "File uploaded!"
  # Hier später: Verarbeiten der hochgeladenen Datei
  print("CSV-Upload ausgelöst – noch keine Verarbeitung implementiert.")

def absa_click_callback(change):
  absa_button.description = "ASBA starts..."
  # Hier beginnt die ABSA
  print("ANALYSE Button geklickt - noch keine Logik implementiert.")


# User Interface
grok_title = widgets.Output()
with grok_title:
    display(HTML("<h2 style='text-align:center; color:#1a73e8;'>Aspect-based sentiment analsis with LLM</h2>"))

prompt_input = widgets.Textarea(
    placeholder="z.B. Whazzup?",
    description="Prompt:",
    layout=widgets.Layout(height="120px", width="100%")
)

ask_grok_button = widgets.Button(
    description="Ask Grok",
    button_style="primary",
    icon="paper-plane"
)
ask_grok_button.on_click(ask_grok)

output_label = widgets.Textarea(
    placeholder="Answer comes here...",
    description="Grok:",
    disabled=True,
    layout=widgets.Layout(height="350px", width="100%")
)

separator = widgets.Output()
with separator:
    display(HTML("<hr>"))  # Trennlinie

csv_analysis_title = widgets.Output()
with csv_analysis_title:
    display(HTML("<h3 style='text-align:center; color:#34a853;'>Aspect-Based Analysis with FileUpload</h3>"))


# CSV Upload Button
upload_button = widgets.Button(
    description="Upload CSV",
    button_style="info",
    icon="upload",
    layout=widgets.Layout(width="48%")
)
upload_button.on_click(uploader_callback)

# Event Button: Start Analysis
absa_button = widgets.Button(
    description="Start Analysis",
    button_style="success",
    icon="play",
    layout=widgets.Layout(width="48%")
)
absa_button.on_click(absa_click_callback)

# Layout für die zwei neuen Buttons nebeneinander
button_row = widgets.HBox([upload_button, absa_button])

# gesamtes UI
ui = widgets.VBox([
    grok_title,
    prompt_input,
    ask_grok_button,
    output_label,
    separator,
    csv_analysis_title,
    button_row
])

# Anzeige des UIs
display(ui)

Schlüssel für Mistral-Large-Instruct speichern

In [17]:
os.environ["MISTRAL_API_KEY"] = userdata.get("MISTRAL_API_KEY")
print("API-Key für Mistral geladen (letzte 4 Zeichen ):", os.environ["MISTRAL_API_KEY"][-4:])

API-Key für Mistral geladen (letzte 4 Zeichen ): fe1a


Systemprompt für Mistral-Large-Instruct

In [30]:
# den API Schlüssel aus der Google Colab Umbegung aufrufen
api_key = os.environ["MISTRAL_API_KEY"]

# Definiere API-Endpunkt & headers
url = "https://chat-ai.academiccloud.de/v1/chat/completions"
headers =  {
    "Authorization": f"Bearer {api_key}",
    "Content_Type": "application/json",
}
# Definiere Nutzdaten (payload) mit Systemprompt
data = {
    "model": "mistral-large-instruct",
    "messages": [
        {"role": "system", "content": "You are Mistral, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."},
        {"role": "user", "content": "Hello does this work? If it works just say 'hell yeah'."}
    ]
}
# Sende die Anfrage
response = requests.post(url, headers=headers, json=data)
# Anfrage ausgeben
print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 401
Response: {'message': 'Unauthorized', 'request_id': '249ca90d01c4e25c267dc018137c5e76'}


In [23]:
print("API Key starts with:", os.environ.get("MISTRAL_API_KEY")[:5])


API Key starts with: 9395f
